In [1]:
import time
import numpy as np

In [2]:
SEED = 42
np.random.seed(SEED)
vector = np.random.randint(0, 5000, size=6_000_000)

In [3]:
def is_prime(n):
    if n < 2 or n % 2 == 0 and n != 2:
        return False
    for i in range(3, int(n**0.5) + 1, 2):
        if n % i == 0:
            return False
    return True

In [4]:
from threading import Thread, Lock

primes = []
lock = Lock()
def get_primes(start, end):
    for i in range(start, end):
        if is_prime(vector[i]):
            with lock:
                primes.append(vector[i])

if __name__ == "__main__":
    N = 8
    size = len(vector) // N
    threads = []

    start_time = time.time()
    for i in range(N):
        start = i*size
        end = start + size
        thread = Thread(target=get_primes, args=(start, end))
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()

    end_time = time.time()
    print("Execution time:", end_time - start_time)

Execution time: 9.721009731292725


In [5]:
print(len(primes))

802855


In [17]:
from concurrent.futures import ProcessPoolExecutor

def get_primes_in_chunk(chunk):
    return [n for n in chunk if is_prime(n)]

if __name__ == "__main__":
    N = 8
    size = len(vector) // N
    chunks = np.array_split(vector, N)

    start_time = time.time()

    with ProcessPoolExecutor(max_workers=N) as executor:
        results = executor.map(get_primes_in_chunk, chunks)

        primes = [prime for chunk in results for prime in chunk]

    end_time = time.time()
    print(end_time - start_time)

5.211313724517822


In [16]:
print(len(primes))

802855
